In [ ]:
pip install torch tensorboard transformers datasets accelerate bitsandbytes peft trl

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging,
                          )
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix,
                             ConfusionMatrixDisplay)
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")

In [ ]:
df = pd.read_csv("hatespeech.csv")
df.head()

In [ ]:
# View only 0 labels in class column
df1 = df[df["class"].isin([2])]
pd.set_option('display.max_colwidth', None)
print(df1)

In [ ]:
model_name = "/content/drive/MyDrive/merged_model_llama3.1"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config,
)

# Turn back on after training
model.config.use_cache = True
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model, tokenizer = setup_chat_format(model, tokenizer)


In [ ]:
# Set up the inference pipeline
pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=1,
                temperature=0.01)

In [ ]:
# Create the prompt for inference
def generate_inference_prompt(sentence):
    return f"""
            Analyze the sentiment of the text enclosed in square brackets,
            determine if it is offensive, hate speech or normal and return the answer as
            the corresponding sentiment label "offensive", "hate", or "normal". Also give a reason for your answer.

            [{sentence}] = """.strip()

In [ ]:
# Example input sentence
input_sentence = "I love watches"

# Generate the inference prompt
inference_prompt = generate_inference_prompt(input_sentence)

# Run the model to generate a prediction
result = pipe(inference_prompt)

# Extract the predicted sentiment from the generated text
generated_text = result[0]['generated_text']
predicted_label = generated_text.split("=")[-1].strip()

print(f'Input sentence: "{input_sentence}"')
print(f'Predicted label: "{predicted_label}"')

In [ ]:
import shutil

# Define source and destination paths
source_dir ='/content/trained_weigths/'
destination_dir = '/content/drive/MyDrive/trained_weights_llama3.1'

# Copy the entire directory
shutil.copytree(source_dir, destination_dir)